# Book recommendation notebook for Model 3

This notebook contains the recommendation algorithm made for model for the book exchange app.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# import books dataset
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/book_recommendations/model_data_updated.csv',encoding='latin-1', error_bad_lines=False)

b'Skipping line 167: expected 9 fields, saw 10\nSkipping line 407: expected 9 fields, saw 10\nSkipping line 436: expected 9 fields, saw 10\nSkipping line 442: expected 9 fields, saw 10\nSkipping line 504: expected 9 fields, saw 10\nSkipping line 519: expected 9 fields, saw 10\nSkipping line 528: expected 9 fields, saw 10\nSkipping line 531: expected 9 fields, saw 10\nSkipping line 634: expected 9 fields, saw 10\nSkipping line 710: expected 9 fields, saw 10\nSkipping line 791: expected 9 fields, saw 10\nSkipping line 840: expected 9 fields, saw 10\nSkipping line 906: expected 9 fields, saw 10\nSkipping line 945: expected 9 fields, saw 10\nSkipping line 962: expected 9 fields, saw 10\nSkipping line 973: expected 9 fields, saw 10\nSkipping line 1025: expected 9 fields, saw 10\nSkipping line 1145: expected 9 fields, saw 10\nSkipping line 1166: expected 9 fields, saw 10\nSkipping line 1171: expected 9 fields, saw 10\nSkipping line 1289: expected 9 fields, saw 10\nSkipping line 1335: expecte

In [4]:
df.head()

,user_id,title,author,publisher,isbn,language,no_of_exchanges,rating,genre
0,UxTG7LfKCWghnwSvMADkSM,HBR 20-Minute Manager Boxed Set (10 Books) (HB...,Harvard Business Review,Harvard Business Review Press,9.781634e+12,English,0,0.0,Business & Economics
1,LngMzaQk8SLRV3RrciP8ew,Financial Accounting Theory,Craig Deegan,McGraw-Hill Education Australia,9.781743e+12,English,3,4.7,Business & Economics
2,HeqaP8onGiXookPed9vATJ,Accounting: An Introduction| 6/E,Peter Atrill,Pearson Higher Education AU,9.781486e+12,English,6,3.8,Business & Economics
3,WdiyuzvrCuE97QgjjMrRuz,The Investment Advisor Body of Knowledge + Tes...,IMCA,John Wiley & Sons,9.781119e+12,English,4,0.0,Business & Economics
4,di2EnkdQ6GaW8arcpPETax,Basics of the U.S. Health Care System: Edition 2,Nancy J. Niles,Jones & Bartlett Learning,9.781284e+12,English,7,2.3,Business & Economics


In [5]:
df.shape

(4458, 9)

In [6]:
# unique user id's in the dataset
df['user_id'].unique()

array(['UxTG7LfKCWghnwSvMADkSM', 'LngMzaQk8SLRV3RrciP8ew',
       'HeqaP8onGiXookPed9vATJ', ..., 'NDsQ2TbfejMmKTf2Erp2h9',
       'X8e2zfA7SxwDM5HpHxZxFx', '5zKFHrjYiQk4diykBrKLUz'], dtype=object)

In [7]:
len(df['user_id'].unique())

4458

In [8]:
df = df.sample(frac = 1)
df.head()

,user_id,title,author,publisher,isbn,language,no_of_exchanges,rating,genre
989,A9jXVWFvn9JdPvtpAHkSaa,Holy Sister (Book of the Ancestor| Book 3),Mark Lawrence,HarperCollins UK,9.780008e+12,English,2,4.7,Fiction
3544,99Wgn5fTCWLYayM8sRzPAK,Native Plants of the Midwest: A Comprehensive ...,Alan Branhagen,Timber Press,9.781605e+12,English,9,4.0,Gardening
1682,39zccF7vSJHP5P5xAjoJET,I| Claudius,Robert Graves,Penguin UK,9.780142e+12,English,7,4.4,Fiction
3768,UWSqb6r33oUBiDJoetkX8o,Heaven Right Here,Lutishia Lovely,Kensington Publishing Corp.,9.780758e+12,English,0,4.4,Fiction
3920,RHA6cBcjysVWUHYqr2ijCy,A Court of Mist and Fury,Sarah J. Maas,Bloomsbury Publishing USA,9.781620e+12,English,6,4.9,Fiction


In [9]:
df.dtypes

user_id              object
 title               object
 author              object
 publisher           object
 isbn               float64
 language            object
 no_of_exchanges      int64
 rating             float64
 genre               object
dtype: object

In [10]:
df.isna().sum()

user_id               0
 title                0
 author             106
 publisher            0
 isbn               363
 language             2
 no_of_exchanges      0
 rating               0
 genre                0
dtype: int64

In [11]:
df.dropna(inplace=True)

In [12]:
df.isna().sum()

user_id             0
 title              0
 author             0
 publisher          0
 isbn               0
 language           0
 no_of_exchanges    0
 rating             0
 genre              0
dtype: int64

In [13]:
from scipy.sparse import csr_matrix
# pivot rating into movie features
mat = df.pivot(index=' isbn',columns='user_id',values=' no_of_exchanges').fillna(0)

In [14]:
mat.head()

user_id,22kKeQT6EftpMcGHwi2tNe,22taMc2ECZX9Q9UNyAzBVV,23X8eMxDKXJmyHwzbCMBdK,23dxZuLdn6gyVmVLeB6bGG,23oArhFY5bH53sYVNRAXwc,24BWYEyJDm34AKVGwCzfbh,24amB9hQneYWzDBb2mZ77Z,25CS2rpHQwChnWS69QDfeu,25QGA5NG6HPBV5r9odJC9R,266p8HTYVgJqLHqeBwrUGC,29ghynqkgyrVnMXNgzd4mU,29ry5387EKixWM9WDJ3eX7,2AYxoN6ySshTf3wBAdLq5q,2AqjhB3TzJwVPT48TQrzuc,2BiCXDjXZj2EqU9mf4c6q9,2C2jiQEDBtvcCwQnFHJdoz,2CsKuh9RnCTpcM3p4jFYGY,2D6B97779USyXKuwXpDqPJ,2DnoYMfL7WkbQo8sAmmoGa,2DsfcVzM5VDNfkdDLKzV3n,2Dwn7dcyC9M7eTfTzYG5aE,2DzD4kotrHfLGKEVenAWRv,2EpbrVxuUm2opQZesmUVzn,2GbB7yYMvJeiRDrR3Wpyip,2HFZoAETzLKZ4TYECVe7nw,2HfENvmjf4g6YmLNoXBfoY,2JBfp46C6JEwoa2jW922jT,2JFB8xJsgxnzpCnDHzCw7s,2JXYKHmG87a5YoLFQWpJYW,2KMM7XvRdtmDjNaB2fkdPA,2LGXgrSZN4vL7gztqrA3zy,2LgKGrozeLBecPmgWyoEko,2MXe4Yu7Sczb7977dfZwLR,2NsXtkQqpdS4gaNogEV4D7,2P29yKfJmfsEADxeKkYQiL,2PCAcuuiWy7HSE7xQxA2YU,2PrDqXgiT2YFqxTd8MNMzr,2PzPNYMcjH9yCrkkFQtLX7,2QNw8i6BoVE8wk9KeCjhdE,2QgiAHcAdLwJHtzf6woAz5,...,o7qGhTDkeLRksaSHhNAQnd,o8UkFhMWbnvRmefPMEr5Kn,o92RfaBTBwmGFDGC55oAL8,oAfgurz8U6cvvNYxe6Amxf,oAsh45UFUUyRhizovwbts2,oAygSdpvtDG5T8wDzuPP4q,oBeTm8E38y9BMqjbzQexDi,oCBrcWnvfemrM5nfG4EKta,oCXZ5esW4dCSXnJuvvSWYX,oCx9DeG5utxixdiVPKi8Mh,oDS3SoMbFA6uz5kGSPob48,oDyHeb9yCy5odHELJRGZ3E,oFZQCwhe3ujUr5WuxvBvQR,oGYUG4yFGsmWK6sswxtn89,oH5Q6ZHi7paSWvDzrEp2oz,oHEZTH6W9toWesyhrYno52,oKA4Kt9DrZDSESWrmEaj29,oLXkzh9BWLjAFyAFT3HuXN,oLigBRxQs4wNn54tcZArx4,oNdnFGuV4XSooPpCCS5qWv,oNkh5AUqStwfFWa88xRmph,oPNEa3wcnjZWQZKGrBK5xa,oPUwMjsokxmwocuVYvk4Rw,oPwwgwvqfhrJnpmi2UHMJG,oQ2MmYTnf2SAZbd5L27dAu,oRjHjMSyPixHGS7gp8RVs7,oRxGvBZj6Xp233coeUkdUC,oSBrE2RMHVNHEvvELEUaGG,oSNaMwLY5Cii9dikdEiWWn,oT8F3FnAX4p99tvuwNLBpV,oThMdJLv9wcZzKggkWQR5X,oTynBiurfYmVvMVwQAR7Xk,oUEzjfdtQDMbduwpevQARk,oUH9LTTk8T9gMDUPpw5JWD,oWNVaxTUF7epyHLLJWFLwi,oWUn3ioNNWyw4LiEMgw97u,oXNb6kR7c8DwCzGfSpdcGC,oXhMa9S4oYhhygrQ7LWDtb,oXvU2LqVcLRcwfP8cCYgsu,oYDXTQyDSPdzokhYUdwA27
isbn,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9.780007e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9.780007e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9.780007e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9.780007e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9.780007e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
mat.shape

(3449, 3987)

In [16]:
sparse_mat = csr_matrix(mat.values)

In [17]:
book_map = df[[' isbn', ' title']]

In [18]:
book_map.head()

,isbn,title
989,9.780008e+12,Holy Sister (Book of the Ancestor| Book 3)
3544,9.781605e+12,Native Plants of the Midwest: A Comprehensive ...
1682,9.780142e+12,I| Claudius
3768,9.780758e+12,Heaven Right Here
3920,9.781620e+12,A Court of Mist and Fury


In [19]:
book_map

,isbn,title
989,9.780008e+12,Holy Sister (Book of the Ancestor| Book 3)
3544,9.781605e+12,Native Plants of the Midwest: A Comprehensive ...
1682,9.780142e+12,I| Claudius
3768,9.780758e+12,Heaven Right Here
3920,9.781620e+12,A Court of Mist and Fury
...,...,...
583,9.781452e+12,The Time Machine
2537,9.781251e+12,Dance with Death
818,9.780571e+12,The Kind Worth Killing
610,9.781445e+12,The 100


In [20]:
book_map.values

array([[9780008152413.0, 'Holy Sister (Book of the Ancestor| Book 3)'],
       [9781604697773.0,
        'Native Plants of the Midwest: A Comprehensive Guide to the Best 500 Species for the Garden'],
       [9780141911748.0, 'I| Claudius'],
       ...,
       [9780571302239.0, 'The Kind Worth Killing'],
       [9781444766899.0, 'The 100'],
       [9781420149340.0, 'Under a Firefly Moon']], dtype=object)

In [21]:
hash_map = {
    movie: i for i,movie in enumerate(list(book_map.set_index(' isbn').loc[mat.index][' title']))
}

In [22]:
hash_map

{'Half of a Yellow Sun': 3,
 'PS| I Love You': 5,
 'Love Rosie (Where Rainbows End)': 8,
 'Thanks for the Memories': 9,
 'Lords of the Bow (Conqueror| Book 2)': 10,
 'HMS Ulysses': 12,
 'San Andreas': 13,
 'Temeraire (The Temeraire Series| Book 1)': 14,
 'The Fellowship of the Ring (The Lord of the Rings| Book 1)': 16,
 'Unfinished Tales': 17,
 'The Lord of the Rings: The Fellowship of the Ring| The Two Towers| The Return of the King': 18,
 'The Hobbit': 19,
 'Wolf Hall: Winner of the Man Booker Prize (The Wolf Hall Trilogy| Book 1)': 20,
 'The Alchemist\x92s Secret (Ben Hope| Book 1)': 21,
 'Mere Christianity': 23,
 'Sharpe\x92s Tiger: The Siege of Seringapatam| 1799 (The Sharpe Series| Book 1)': 24,
 'The Last Kingdom (The Last Kingdom Series| Book 1)': 25,
 'Remarkable Creatures': 26,
 'A Foreign Country (Thomas Kell Spy Thriller| Book 1)': 27,
 'The Book of Lost Tales 1 (The History of Middle-earth| Book 1)': 28,
 'The Demon King (The Seven Realms Series| Book 1)': 29,
 'The Homeco

In [23]:
!pip install fuzzywuzzy

In [24]:
# function for searching books in database using keywords
from fuzzywuzzy import fuzz 
def matching(book_map, fav_book):
  """
  returns the closest match via fuzzy ratio
  If no match found, returns None

  Parameters
  --------
  book_map : dataframe of book titles along with their isbn
  fav_book : name of the favourite book

  Return
  -------
  index of closest match
  """
  match = []
  # get match
  for isbn, title in book_map.items():
    ratio = fuzz.ratio(title.lower(),fav_book.lower())
    if ratio >=60:
      match.append((title,isbn,ratio))
    # sort
  match = sorted(match,key = lambda x:x[2])[::-1]
  if not match:
    print("No match found!!!")
  else:
    print("Found possible matches in our database: {0}".format([x[0] for x in match]))
    return match

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [25]:
book_dict = dict(zip(book_map[' isbn'],book_map[' title']))

In [26]:
matching(book_dict,"Requiem for Dream")[0][0]

Found possible matches in our database: ['Requiem for a Dream: A Novel']


'Requiem for a Dream: A Novel'

In [27]:
# function for making recommendations
import time
from sklearn.neighbors import NearestNeighbors
def recommend(fav_book,n_recommendations,book_map,data):
  """
  Returns the top n recommendations based on the keyword entered

  Parameters
  ---------
  fav_book : Keyword for favourite book
  n_recommendations : Number of recommendations to be made
  book_map : book database with title and isbn
  data : csr matrix of user_id, ratings and isbn

  Return
  --------
  list of top n book recommendations
  """
  # instantiating model
  model = NearestNeighbors(algorithm ='brute',metric='cosine')
  # fitting the model
  model.fit(data)
  # get input book isbn
  book_isbn = matching(book_map, fav_book)
  # inference
  print("Recommendation system starting to make inference......")
  t0 = time.time()
  distances, isbns = model.kneighbors(
      data[book_isbn],
      n_neighbors=n_recommendations+1
  )
  recommends = sorted(list(zip(isbns.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x : x[1])[:0:-1]
  print(f"Time Taken for recommendation : {time.time()-t0}")
  return recommends 

In [28]:
# function to show movie names after recommedations
def make_recommendations(fav_book, n_recommendations, mat, hash_map):
  """
  makes n recommendations based on fav books
  Parameters
  ----------
  fav_book : favourite book
  n_recommendation : number of recommedations to be made
  Return
  ----------
  None, print recommendations
  """
  raw_recommendations = recommend(fav_book,n_recommendations=10,book_map=hash_map, data=mat)
  reverse_hashmap = {v : k for k,v in hash_map.items}
  print('Recommendations for {}:'.format(fav_book))
  for i,(isbn,dist) in enumerate(raw_recommendations):
    print('{0}: {1}, with distance of {2}'.format(i+1,reverse_hashmap[isbn],dist))

In [29]:
book_map

,isbn,title
989,9.780008e+12,Holy Sister (Book of the Ancestor| Book 3)
3544,9.781605e+12,Native Plants of the Midwest: A Comprehensive ...
1682,9.780142e+12,I| Claudius
3768,9.780758e+12,Heaven Right Here
3920,9.781620e+12,A Court of Mist and Fury
...,...,...
583,9.781452e+12,The Time Machine
2537,9.781251e+12,Dance with Death
818,9.780571e+12,The Kind Worth Killing
610,9.781445e+12,The 100


In [30]:
book_map[book_map[' isbn']==9781407097626]

,isbn,title
840,9.781407e+12,The Lost World


In [31]:
print(sparse_mat[1])

  (0, 935)	1.0


In [32]:
hash_map

{'Half of a Yellow Sun': 3,
 'PS| I Love You': 5,
 'Love Rosie (Where Rainbows End)': 8,
 'Thanks for the Memories': 9,
 'Lords of the Bow (Conqueror| Book 2)': 10,
 'HMS Ulysses': 12,
 'San Andreas': 13,
 'Temeraire (The Temeraire Series| Book 1)': 14,
 'The Fellowship of the Ring (The Lord of the Rings| Book 1)': 16,
 'Unfinished Tales': 17,
 'The Lord of the Rings: The Fellowship of the Ring| The Two Towers| The Return of the King': 18,
 'The Hobbit': 19,
 'Wolf Hall: Winner of the Man Booker Prize (The Wolf Hall Trilogy| Book 1)': 20,
 'The Alchemist\x92s Secret (Ben Hope| Book 1)': 21,
 'Mere Christianity': 23,
 'Sharpe\x92s Tiger: The Siege of Seringapatam| 1799 (The Sharpe Series| Book 1)': 24,
 'The Last Kingdom (The Last Kingdom Series| Book 1)': 25,
 'Remarkable Creatures': 26,
 'A Foreign Country (Thomas Kell Spy Thriller| Book 1)': 27,
 'The Book of Lost Tales 1 (The History of Middle-earth| Book 1)': 28,
 'The Demon King (The Seven Realms Series| Book 1)': 29,
 'The Homeco

In [33]:
len(hash_map)

3417

In [34]:
# function to get list of recommmendations based on isbn
def recommendations_from_isbn(isbn,data,n_neighbors,book_dict,hash_map):
  """
  This function provides recommendations based on isbn
  
  Parameters
  ---------
  isbn: isbn of the book from which recommendations have to be made
  data: sparse matrix data
  n_neighbors: number of neighbors for knn models
  book_dict: dictionary linking isbn to book name
  hash_map: dictionary linking title to sparse matrix index

  Return
  --------
  recoms: recommendations from the passed isbn
  """
  dummy_model = NearestNeighbors(algorithm='brute',metric='cosine')
  dummy_model.fit(data)
  title = book_dict[isbn]
  r,d = dummy_model.kneighbors(data[hash_map[title]],n_neighbors=11)
  recoms = []
  recommends = sorted(list(zip(d.squeeze().tolist(),r.squeeze().tolist())),key=lambda x : x[1])[:0:-1]
  for (i,j) in recommends:
    for title, isbn in hash_map.items():
      if isbn == i:
        recoms.append(title)
  return recoms

In [35]:
recommendations_from_isbn(9780984835935.0,data=sparse_mat,n_neighbors=10, book_dict=book_dict, hash_map=hash_map)

['Wool',
 'The Excalibur Codex: An explosive historical thriller that will have you on the edge of your seat',
 "War of Don Emmanuel's Nether Parts",
 "The Curious Incident of the Dog in the Night-time: Vintage Children's Classics",
 'Casino Royale',
 'The Name of the Rose']

In [36]:
# function to get recommended books' isbn
def get_books_isbn(isbn,data,n_neighbors,book_dict,hash_map):
  """
  Function to return isbns of recommended books from the isbn passed

  Parameters
  ---------
  isbn: isbn of the book from which recommendations have to be made
  data: sparse matrix data
  n_neighbors: number of neighbors for knn models
  book_dict: dictionary linking isbn to book name
  hash_map: dictionary linking title to sparse matrix index

  Return
  --------
  isbns: isbns of recommendations from the passed isbn

  """
  recoms = recommendations_from_isbn(isbn,data=data,n_neighbors=n_neighbors,book_dict=book_dict,hash_map=hash_map)
  isbns = []
  for book_name in recoms:
    for isb,title in book_dict.items():
      if title == book_name:
        isbns.append(isb)
  return isbns

In [38]:
get_books_isbn(9781407097626,data=sparse_mat,n_neighbors=10, book_dict=book_dict, hash_map=hash_map)

[9781448137749.0,
 9781448150205.0,
 9781448136285.0,
 9781448127627.0,
 9781448139859.0,
 9781448139248.0]

In [39]:
# Making recommendations using only one base model
base_model = NearestNeighbors(algorithm='brute', metric='cosine')
base_model.fit(sparse_mat)
title = book_dict[9781448139859.0]
r,d = base_model.kneighbors(sparse_mat[hash_map[title]],n_neighbors=11)
recoms = []
recommends = sorted(list(zip(d.squeeze().tolist(),r.squeeze().tolist())),key=lambda x : x[1])[:0:-1]
for (i,j) in recommends:
  for title, isbn in hash_map.items():
    if isbn == i:
      recoms.append(title)
recoms

['Star Wars: Darth Plagueis',
 'The Ultimate Slow Cooker Cookbook: Over 100 delicious| fuss-free recipes - from family favourites to dishes for a dinner party',
 'The Excalibur Codex: An explosive historical thriller that will have you on the edge of your seat',
 "War of Don Emmanuel's Nether Parts",
 'The Name of the Rose',
 'Casino Royale']

In [40]:
isbns = []
for book_name in recoms:
  for isb,title in book_dict.items():
    if title == book_name:
      isbns.append(isb)
isbns

[9781448136285.0,
 9781448146093.0,
 9781448127627.0,
 9781448113798.0,
 9781448137749.0,
 9781448139248.0]

In [41]:
# Making a base model and training it
model = NearestNeighbors(metric='cosine',algorithm='brute')
model.fit(sparse_mat)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [67]:
# Function for getting book title using base model
def get_title(model,isbn, book_dict=book_dict, data=sparse_mat, hash_map=hash_map):
  """
  Function to return title of recommended books using isbn

  Parameters
  -----------
  model : base model globally trained to recommend books
  isbn : isbn of book from which recommendations are to be made

  Return
  -----------
  recoms : list of titles of recommended books
  """
  title = book_dict[isbn]
  r,d = base_model.kneighbors(data[hash_map[title]],n_neighbors=11)
  recoms = []
  recommends = sorted(list(zip(d.squeeze().tolist(),r.squeeze().tolist())),key=lambda x : x[1])[:0:-1]
  for (i,j) in recommends:
    for title, isbn in hash_map.items():
      if isbn == i:
        recoms.append(title)
  return recoms

In [43]:
get_title(model,isbn=9781448139859.0)

['Star Wars: Darth Plagueis',
 'The Ultimate Slow Cooker Cookbook: Over 100 delicious| fuss-free recipes - from family favourites to dishes for a dinner party',
 'The Excalibur Codex: An explosive historical thriller that will have you on the edge of your seat',
 "War of Don Emmanuel's Nether Parts",
 'The Name of the Rose',
 'Casino Royale']

In [68]:
# Function to get isbn from title
def get_recommendations(model,isbn, book_dict=book_dict, data=sparse_mat, hash_map=hash_map):
  """
  Function to isbn of recommended books by using a globally declared model

  Parameters
  ----------
  model - globally trained model for recommendations
  isbn - isbn of book from which recommendations are to be made

  Return
  ----------
  isbns - list of recommended books' isbns
  """
  recoms = get_title(model,isbn)
  isbns = []
  for book_name in recoms:
    for isb,title in book_dict.items():
      if title == book_name:
        isbns.append(isb)
  return isbns

In [45]:
get_recommendations(model,isbn=9781448139859.0)

[9781448136285.0,
 9781448146093.0,
 9781448127627.0,
 9781448113798.0,
 9781448137749.0,
 9781448139248.0]

In [46]:
# Export model pkl file
import pickle as pkl
pkl.dump(model,open('/content/drive/MyDrive/book_recommendations/model_file.pkl','wb'))

In [63]:
# Recommending books of a particular genre
genre_df = df[df[' genre']=='Fiction ']
gen_mat = genre_df.pivot(index=' isbn',columns='user_id',values=' no_of_exchanges').fillna(0)
gen_sparse_mat = csr_matrix(gen_mat.values)

gen_book_map = genre_df[[' isbn', ' title']]
gen_hash_map = {
    book: i for i,book in enumerate(list(gen_book_map.set_index(' isbn').loc[gen_mat.index][' title']))
}
gen_book_dict = dict(zip(gen_book_map[' isbn'],gen_book_map[' title']))

In [64]:
get_books_isbn(9781407097626,data=gen_sparse_mat,n_neighbors=10, book_dict=gen_book_dict, hash_map=gen_hash_map)

[9781446486474.0,
 9781446485279.0,
 9781446474679.0,
 9781446484203.0,
 9781446484197.0,
 9781446485002.0]

In [66]:
genre_df[genre_df[' isbn']==9781407097626]

,user_id,title,author,publisher,isbn,language,no_of_exchanges,rating,genre
840,c89hVBK7R5nXx5xRwfPaqU,The Lost World,Michael Crichton,Random House,9.781407e+12,English,6,4.4,Fiction


In [69]:
get_recommendations(model,isbn=9781448139859.0, book_dict=gen_book_dict, data=gen_sparse_mat, hash_map=gen_hash_map)

[9781448136285.0,
 9781448127627.0,
 9781448113798.0,
 9781448137749.0,
 9781448139248.0]